<a href="https://colab.research.google.com/github/Hamza-Ghnaim/Design-and-Implementation-of-a-Robotic-Goalie/blob/main/Copy_of_OpenInsights_DS_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Open Insights Data Science - CTU 13 Botnet Detection Task**

>**Description**: The goal of this assignment is to build a machine learning model that can predict botnet traffic. The dataset used is "*CTU-13*" which captures real botnet traffic mixed with normal and background traffics. The dataset consists of 13 captures (scenarios) that differ in the executed malware. In this assignment, you will be using scenario 5.

>The assignment is divided into 7 tasks that cover data analysis, feature extraction, and model building. Follow the instructions within each task to complete the assignment.

>We are looking for code that is clear, readable, and executable. While it is recommended to complete the entire assignment, incomplete assignments are also accepted. 
>
>**Objective**: Build an XGBOOST machine learning model to predict botnet traffic
>
>**Dataset**: CTU 13 - Scenario 5 [https://www.stratosphereips.org/datasets-ctu13]
>
>**Please complete all cells marked as "Task", until you reach a completed running XGBoost model that predicts botnet traffic**

In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import time, itertools, numpy as np, xgboost as xgb,  pandas as pd, math, glob, json, operator, requests
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Data Ingestion
# Ingest scenario 5 of the CTU-13 dataset
# The Scenario is captured as a pandas dataframe
!gdown --id 1p0ZMJTZIV5cIpuS8tQ-UgNeABDqBi2XJ
scenario_5 = pd.read_csv("ctu13_scenario5.csv")
scenario_5["Scenario"] = 5

Downloading...
From: https://drive.google.com/uc?id=1p0ZMJTZIV5cIpuS8tQ-UgNeABDqBi2XJ
To: /content/ctu13_scenario5.csv
100% 17.8M/17.8M [00:00<00:00, 218MB/s]


In [ ]:
# Task 1 - Data Cleansing & Standardization
# Step 1. Impute Null and NaN values
# Step 2. Convert Hex values of "Sport" and "Dport" to Integers
# Step 3. Parse "StartTime" into a datetime format

# Task 1: Data Cleansing and Standarization

In [129]:
# first of all we create a copy of our dataframe and perform the changes on it

df_copy = scenario_5.copy()

In [130]:
# we take a look at our data

df_copy.tail()

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label,Scenario
129827,2011/08/15 17:13:26.665269,0.020175,tcp,147.32.84.59,1121,->,147.32.80.13,80,SPA_SPA,0.0,0.0,5,1070,726,flow=To-Background-CVUT-Proxy,5
129828,2011/08/15 17:13:26.715751,0.000306,udp,147.32.86.20,63044,<->,147.32.80.9,53,CON,0.0,0.0,2,261,84,flow=To-Background-UDP-CVUT-DNS-Server,5
129829,2011/08/15 17:13:26.716285,0.076739,tcp,147.32.86.20,1311,->,188.138.84.239,80,SPA_SA,0.0,0.0,10,3639,429,flow=Background-TCP-Established,5
129830,2011/08/15 17:13:26.721279,0.000244,tcp,78.94.167.118,65017,->,147.32.84.118,6881,S_RA,0.0,0.0,2,134,74,flow=Background-TCP-Attempt,5
129831,2011/08/15 17:13:26.758894,0.000551,udp,178.90.228.239,57645,<->,147.32.84.229,13363,CON,0.0,0.0,2,132,72,flow=Background-UDP-Established,5


In [131]:
# now we display a brief description of the data

df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129832 entries, 0 to 129831
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   StartTime  129832 non-null  object 
 1   Dur        129832 non-null  float64
 2   Proto      129832 non-null  object 
 3   SrcAddr    129832 non-null  object 
 4   Sport      129035 non-null  object 
 5   Dir        129832 non-null  object 
 6   DstAddr    129832 non-null  object 
 7   Dport      129455 non-null  object 
 8   State      129832 non-null  object 
 9   sTos       128974 non-null  float64
 10  dTos       122936 non-null  float64
 11  TotPkts    129832 non-null  int64  
 12  TotBytes   129832 non-null  int64  
 13  SrcBytes   129832 non-null  int64  
 14  Label      129832 non-null  object 
 15  Scenario   129832 non-null  int64  
dtypes: float64(3), int64(4), object(9)
memory usage: 15.8+ MB


In [132]:
# we calculate the number of null values in each feature

df_copy.isnull().sum()

StartTime       0
Dur             0
Proto           0
SrcAddr         0
Sport         797
Dir             0
DstAddr         0
Dport         377
State           0
sTos          858
dTos         6896
TotPkts         0
TotBytes        0
SrcBytes        0
Label           0
Scenario        0
dtype: int64

In [133]:
#normally we drop the columns with missing values more than or equal to 40% , but no such columns exist in this dataset

null_cols = [column for column in df_copy.columns if abs(df_copy[column].isnull().sum()/(len(df_copy.iloc[:,])) * 100) >= 40]
null_cols

[]

In [134]:
# now we fill the missing values in each column with the most frequent value(mode) of the column

def Impute_cols(dataframe):

  """  this function fills the missing values of numerical and categorical columns in a dataframe with the mode of the column  """

  # first we save columns with missing values in a list
  null_cols  =[column for column in dataframe.columns if dataframe[column].isnull().any()]

  # now we loop through the list of columns with nulls
  for column in null_cols:

    # count the most frequent value of each column and save it to a dataframe
    mode = dataframe[column].value_counts().index[0]

    #fill in with the mode
    dataframe[column].fillna(mode,inplace = True)

  return dataframe
  
# another alternative for the above function is sklearn's SimpleImputer

In [135]:
#call the function
df_copy = Impute_cols(df_copy)

In [ ]:
df_copy.isnull().sum()

In [136]:
# Now we deal with the Hexa decimal values from the columns "Sport" and "Dport"

df_copy['Dport']= df_copy['Dport'].map(lambda cell: str(int(cell,0)))
df_copy['Sport']= df_copy['Sport'].map(lambda cell: str(int(cell,0)))

In [137]:
# Now we parse the"StartTime" column into a datetime format

In [138]:
df_copy["StartTime"] .head()

0    2011/08/15 16:43:28.078942
1    2011/08/15 16:43:32.283576
2    2011/08/15 16:43:32.456441
3    2011/08/15 16:43:32.850648
4    2011/08/15 16:45:09.305002
Name: StartTime, dtype: object

In [139]:
df_copy["StartTime"] = pd.to_datetime(df_copy["StartTime"], format="%Y/%m/%d %H:%M:%S.%f")

In [140]:
df_copy["StartTime"] .head()

0   2011-08-15 16:43:28.078942
1   2011-08-15 16:43:32.283576
2   2011-08-15 16:43:32.456441
3   2011-08-15 16:43:32.850648
4   2011-08-15 16:45:09.305002
Name: StartTime, dtype: datetime64[ns]

# Task 1 done

In [ ]:
# Task 2 - Data Labeling
# Step 1. Create a function that remaps the "Label" Column to three values "Background", "Normal" and "Botnet"
# Step 2. Apply the function on the "Label" Column

# Task 2:Data Labeling

In [143]:
df_copy.Label.value_counts()

flow=To-Background-UDP-CVUT-DNS-Server                      50817
flow=Background-UDP-Established                             37203
flow=Background-TCP-Established                             15156
flow=Background-Established-cmpgw-CVUT                       9331
flow=Background                                              3426
                                                            ...  
flow=From-Botnet-V46-TCP-HTTP-Google-Net-Established-6          1
flow=From-Botnet-V46-TCP-Established-Custom-Encryption-8        1
flow=From-Botnet-V46-TCP-Established-HTTP-Ad-4                  1
flow=From-Botnet-V46-TCP-CC7-Custom-Encryption                  1
flow=From-Botnet-V46-TCP-Established                            1
Name: Label, Length: 77, dtype: int64

In [ ]:
# Task 3 - Exploratory Data Analysis
# Step 1. Calculate the distribution of the "Label" ( Background, Botnet, Normal )
# Step 2. Visualize the distribution of the following Columns "Dur", "Proto", "Dir", "sTos", "dTos", "Sport", "Dport", "State", "TotPkts", "TotBytes", "SrcBytes"

In [ ]:
# Task 4 - Datetime Feature Enrichment
time_cols = ["hod", "dow", "wom", "moy"]
# Step 1. Extract hour of day (hod) from StartTime
# Step 2. Extract day of week (dow) from StartTime
# Step 3. Extract week of month (wom) from StartTime
# Step 4. Extract month of year (moy) from StartTime

In [ ]:
# Task 5 - One Hot Encoding & Scaling / Normalization
# Step 1. Perform One-Hot Encoding on the following columns ["Proto", "Dir", "State", "sTos", "dTos"]
ohe_in_cols = ["Proto", "Dir", "State", "sTos", "dTos"]
# Step 2. Perform scaling using StandardScaler on the following columns ["Dur", "TotPkts", "TotBytes", "SrcBytes"]
numerical_cols = ["Dur", "TotPkts", "TotBytes", "SrcBytes"]

In [ ]:
# Task 6 - Label Encoding
# Step 1. Define a function to remap "Label" column into integers as follows Background = 0, Normal = 1, Botnet = 2
# Step 2. Apply function to "Label" column

In [ ]:
# Task 7 - Model Build
feature_cols = numerical_cols + time_cols + list(ohe_out_cols)
# Step 1. Define x,y based on the feature_cols and the label_cols
# Step 2. Perform a 80:20 train-test split 
# Step 3. Initialize an XGBoost Classifier Model
# Step 4. Train Model on the training dataset produced by step 3
# Step 5. Predict on the testing dataset produced by step 3
# Step 6. Calculate f1_score, recall, precision for the testing dataset predictions